In [9]:
import jax_spline as spline
from jax_spline_test import *

spline creation, forward/reverse sensitivity check
adj error: 0 

spline evaluation, forward/reverse sensitivity check
adj error: -2.25514e-17 



In [507]:
import jax
import jax.numpy as jnp
import jax.random as random
from jax import jvp, vjp
import numpy as np
jax.config.update("jax_enable_x64", True)

In [500]:
N = 4
s1 = jnp.linspace(0.0,1.0,N)  # spline points
s2 = jnp.linspace(0.0,1.0,N)  # spline points
key = random.key(123456789)

key, sub = random.split(key); 
#x = jnp.array([[(3.0*s_1**2 - 2.0*s_1**3)*(3.0*s_2**2 - 2.0*s_2**3) for s_2 in s2] for s_1 in s1]) 
#x = jnp.outer((3.0*s1**2 - 2.0*s1**3),(3.0*s2**2 - 2.0*s2**3)) 
f = random.normal(sub,(N,N))

In [501]:
f

Array([[-0.92025661, -0.24530095,  0.04830577, -1.1242524 ],
       [ 0.56743798, -0.11259252,  1.48550824, -1.30495303],
       [-1.06828365, -0.03066559,  0.70936112, -0.37517106],
       [ 0.20228566, -2.82956798,  1.96863616,  0.35150377]],      dtype=float64)

In [502]:
ss_1 = np.random.rand()  # uniform on (0,1)
ss_2 = np.random.rand()  # uniform on (0,1)
i = jnp.sum(jnp.where(s1<ss_1,1,0))
j = jnp.sum(jnp.where(s2<ss_2,1,0))
print(i,j,f[i][j])

# assert 0<i and i<s1.size-1
# assert 0<j and j<s2.size-1

key, sub = random.split(key); f_d  = random.normal(sub,(N,N))
key, sub = random.split(key); f_b  = random.normal(sub,(N,N))
key, sub = random.split(key); fp_b = random.normal(sub,(N,N))

3 1 -2.829567982139835


In [38]:
f_b

Array([[-0.89510267,  1.40890145,  0.12466555,  0.125378  ],
       [ 0.19611859, -1.86656033, -0.2046865 , -1.40192039],
       [ 0.32809385, -0.43845637,  0.74564719,  1.14399845],
       [-0.77933602,  0.39265631,  1.30420188,  0.39797021]],      dtype=float64)

specific value check

In [488]:
ss_1=(s1[2]+s1[3])/2
ss_2=(s2[0]+s2[1])/2

In [40]:
(3.0*ss_1**2 - 2.0*ss_1**3)*(3.0*ss_2**2 - 2.0*ss_2**3)

Array(0.03703704, dtype=float64)

# original combined evaluation

evaluate every point 

In [57]:
def x_y(x,y,u,a,b):
    u_a = jnp.zeros_like(y)
    u_y_ = jnp.zeros_like(y)
    u_x = jnp.zeros_like(u)
    u_y = jnp.zeros_like(u)
    u_xy = jnp.zeros_like(u)
    #calculating derivatives
    for j in range(y.size):        
        u_y = u_y.at[:,j].set(spline.derivs(y,u[:,j]))
    for i in range(x.size):
        u_x = u_x.at[i,:].set(spline.derivs(x,u[i,:]))
        u_xy = u_xy.at[i,:].set(spline.derivs(x,u_y[i,:]))
    #splining by fixing y
    for j in range(y.size):
        u_a = u_a.at[j].set(spline.eval(x,u[j,:],u_x[j,:],a))
        u_y_ = u_y_.at[j].set(spline.eval(x,u_y[j,:],u_xy[j,:],a))
    #fixing x
    u_ab = spline.eval(y,u_a,u_y_,b)
    return u_ab

In [58]:
x_y(s1,s2,f,ss_1,ss_2)

Array(-1.10123204, dtype=float64)

In [59]:
def fun(u):
    return x_y(s1,s2,u,ss_1,ss_2)

In [60]:
ff_b = np.random.rand() 
_, u_d = jvp(fun, (f,), (f_d,))
_, u_barfunc = vjp(fun, f)
u_bar = u_barfunc(ff_b)
print('adj error: %g \n' % (dp(u_bar[0],f_d)-dp(ff_b,u_d)))

adj error: 1.11022e-16 



In [61]:
def y_x(x,y,u,a,b):
    #fixing x
    u_b = jnp.zeros_like(x)
    u_x_ = jnp.zeros_like(x)
    u_y = jnp.zeros_like(u)
    u_x = jnp.zeros_like(u)
    u_xy = jnp.zeros_like(u)
    for i in range(x.size):
        u_x = u_x.at[i,:].set(spline.derivs(x,u[i,:]))
    for j in range(y.size):        
        u_y = u_y.at[:,j].set(spline.derivs(y,u[:,j]))
        u_xy = u_xy.at[:,j].set(spline.derivs(y,u_x[:,j]))
    for i in range(x.size):
        u_b = u_b.at[i].set(spline.eval(y,u[:,i],u_y[:,i],b))
        u_x_ = u_x_.at[i].set(spline.eval(y,u_x[:,i],u_xy[:,i],b))
    #fixing y
    u_ab = spline.eval(x,u_b,u_x_,a)
    return u_ab

In [62]:
y_x(s1,s2,f,ss_1,ss_2)

Array(-1.10123204, dtype=float64)

In [63]:
def fun(u):
    return y_x(s1,s2,u,ss_1,ss_2)

In [64]:
ff_b = np.random.rand() 
_, u_d = jvp(fun, (f,), (f_d,))
_, u_barfunc = vjp(fun, f)
u_bar = u_barfunc(ff_b)
print('adj error: %g \n' % (dp(u_bar[0],f_d)-dp(ff_b,u_d)))

adj error: -5.55112e-17 



# split creation and evaluation

In [41]:
def creation_xy(x,y,u):
    u_y = jnp.zeros_like(u)
    u_x = jnp.zeros_like(u)
    u_xy = jnp.zeros_like(u)
    #calculating derivatives
    for j in range(y.size):        
        u_y = u_y.at[:,j].set(spline.derivs(y,u[:,j]))
    for i in range(x.size):
        u_x = u_x.at[i,:].set(spline.derivs(x,u[i,:]))
        u_xy = u_xy.at[i,:].set(sp line.derivs(x,u_y[i,:]))
    return u_x,u_y, u_xy

def creation_yx(x,y,u):
    u_y = jnp.zeros_like(u)
    u_x = jnp.zeros_like(u)
    u_xy = jnp.zeros_like(u)
    for i in range(x.size):
        u_x = u_x.at[i,:].set(spline.derivs(x,u[i,:]))
    for j in range(y.size):        
        u_y = u_y.at[:,j].set(spline.derivs(y,u[:,j]))
        u_xy = u_xy.at[:,j].set(spline.derivs(y,u_x[:,j]))
    return u_x,u_y, u_xy

In [154]:
N = 4
x = jnp.linspace(0.0,1.0,N)  # spline points
y = jnp.linspace(0.0,1.0,N)  # spline points
key = random.key(123456789)

key, sub = random.split(key); 

In [44]:
key, sub = random.split(key); u    = random.normal(sub,(N,N))
key, sub = random.split(key); u_d  = random.normal(sub,(N,N))
key, sub = random.split(key); u_b  = random.normal(sub,(N,N))

# compute forward and reverse sensitivities for spline creation
_, forward_d = jvp(creation_xy, (x, y, u), (jnp.zeros_like(x), jnp.zeros_like(y), u_d))


_, f_vjp = vjp(creation_xy,x,y,u)
reverse_b = f_vjp((jnp.ones_like(u),jnp.ones_like(u),jnp.ones_like(u)))

print('spline creation, forward/reverse sensitivity check')
print('adj error: %g \n' % (sum([dp(jnp.ones_like(u),i) for i in forward_d]) - dp(reverse_b[2],u_d)))

spline creation, forward/reverse sensitivity check
adj error: 0 



In [45]:
# compute forward and reverse sensitivities for spline creation
_, forward_d = jvp(creation_yx, (x, y, u), (jnp.zeros_like(x), jnp.zeros_like(y), u_d))


_, f_vjp = vjp(creation_yx,x,y,u)
reverse_b = f_vjp((jnp.ones_like(u),jnp.ones_like(u),jnp.ones_like(u)))

print('spline creation, forward/reverse sensitivity check')
print('adj error: %g \n' % (sum([dp(jnp.ones_like(u),i) for i in forward_d]) - dp(reverse_b[2],u_d)))

spline creation, forward/reverse sensitivity check
adj error: 7.10543e-15 



In [46]:
# Compute the outputs of both derivatives
output_xy = creation_xy(x, y, f)
output_yx = creation_yx(x, y, f)

# Check if the outputs are the same
def check_outputs(output1, output2):
    for o1, o2 in zip(output1, output2):
        if not jnp.allclose(o1, o2):
            return False
    return True

outputs_same = check_outputs(output_xy, output_yx)

# Print the results
print("Output of creation_xy:", output_xy)
print("Output of creation_yx:", output_yx)
print("Outputs are the same:", outputs_same)

Output of creation_xy: (Array([[-0.        ,  2.85192058, -2.69062091,  0.        ],
       [ 0.        ,  2.91878493, -3.41250738,  0.        ],
       [-0.        ,  4.47305071, -1.89339998,  0.        ],
       [ 0.        ,  2.33059815,  6.57476189,  0.        ]],      dtype=float64), Array([[ 0.        , -0.        ,  0.        ,  0.        ],
       [-0.13617351,  2.14531014,  1.29665608,  0.80392115],
       [-0.78754935, -6.64952233,  0.7628738 ,  3.52604751],
       [ 0.        ,  0.        ,  0.        ,  0.        ]],      dtype=float64), Array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  4.2436244 , -4.07903134,  0.        ],
       [ 0.        , -2.38432634, 23.49111371,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]],      dtype=float64))
Output of creation_yx: (Array([[-0.        ,  2.85192058, -2.69062091,  0.        ],
       [ 0.        ,  2.91878493, -3.41250738,  0.        ],
       [-0.        ,  4.47305071

## modified evaluation, only use the four points around the evaluated one

In [478]:
def eval_reverse(s, x, x_b, xp, xp_b, ss, ss_b, xx_b):
    i = np.sum(np.where(s<ss,1,0));

    if (i==0):         # outside entire interval
        x_b = x_b.at[0].set(x_b[0] + xx_b)

    elif (i==s.size):  # outside entire interval
         x_b = x_b.at[-1].set(x_b[-1] + xx_b)
            
    else:              # interval (s[i-1], s[i])
        ds   = s[i]-s[i-1]
        t    = (ss-s[i-1]) / ds
        x_b = x_b.at[i-1].set(x_b[i-1]  + (1   - 3*t**2 + 2*t**3) * xx_b)
        x_b = x_b.at[i].set(x_b[i]    + (      3*t**2 - 2*t**3) * xx_b)
        xp_b = xp_b.at[i-1].set(xp_b[i-1] + (  t - 2*t**2 +   t**3) * xx_b*ds)
        xp_b = xp_b.at[i].set(xp_b[i]   + (    -   t**2 +   t**3) * xx_b*ds)
        ss_b = ss_b + ( (  - 6*t + 6*t**2) * x[i-1]      \
                      + (    6*t - 6*t**2) * x[i]        \
                      + (1 - 4*t + 3*t**2) * xp[i-1]*ds  \
                      + (  - 2*t + 3*t**2) * xp[i]  *ds ) * xx_b/ds;

    return x_b, xp_b, ss_b

def eval_forward(s, x, x_d, xp, xp_d, ss, ss_d):
    i = np.sum(np.where(s<ss,1,0));

    if (i==0):         # outside entire interval
        xx_d = x_d[0]

    elif (i==s.size):  # outside entire interval
        xx_d = x_d[-1]

    else:              # interval (s[i-1], s[i])
        ds   = s[i]-s[i-1]
        t    = (ss-s[i-1]) / ds
        t_d  =  ss_d / ds
        xx_d = (1   - 3*t**2 + 2*t**3) * x_d[i-1]     \
             + (      3*t**2 - 2*t**3) * x_d[i]       \
             + (  t - 2*t**2 +   t**3) * ds*xp_d[i-1] \
             + (    -   t**2 +   t**3) * ds*xp_d[i]   \
             + ( (  - 6*t + 6*t**2) * x[i-1]          \
               + (    6*t - 6*t**2) * x[i]            \
               + (1 - 4*t + 3*t**2) * ds*xp[i-1]      \
               + (  - 2*t + 3*t**2) * ds*xp[i] ) * t_d

    return xx_d

def eval_2p(s, x, xp, ss):

    if x[0]==x[1]:         # outside entire interval 
        xx = x[-1]

    else:      func        # interval (s[i-1], s[i])
        ds = s[1]-s[0]
        t  = (ss-s[0]) / ds
        xx = (1   - 3*t**2 + 2*t**3) * x[0]     \
           + (      3*t**2 - 2*t**3) * x[1]   \
           + (  t - 2*t**2 +   t**3) * ds*xp[0] \
           + (    -   t**2 +   t**3) * ds*xp[1]
        

    return xx

def eval_2p_reverse(s, x, xp, ss, xx_b, x_b, xp_b, ss_b):

    if x[0]==x[1]:         # outside entire interval 
        x_b = x_b.at[-1].set(x_b[-1] + xx_b)

    else:              # interval (s[i-1], s[i])
        ds = s[1]-s[0]
        t  = (ss-s[0]) / ds
        x_b = x_b.at[0].set(x_b[0]  + (1   - 3*t**2 + 2*t**3) * xx_b)
        x_b = x_b.at[1].set(x_b[1]    + (      3*t**2 - 2*t**3) * xx_b)
        xp_b = xp_b.at[0].set(xp_b[0] + (  t - 2*t**2 +   t**3) * xx_b*ds)
        xp_b = xp_b.at[1].set(xp_b[1]   + (    -   t**2 +   t**3) * xx_b*ds)
        ss_b = ss_b + ( (  - 6*t + 6*t**2) * x[0]      \
                      + (    6*t - 6*t**2) * x[1]        \
                      + (1 - 4*t + 3*t**2) * xp[0]*ds  \
                      + (  - 2*t + 3*t**2) * xp[1]  *ds ) * xx_b/ds
    return x_b, xp_b, ss_b

def eval_2p_forward(s, x, x_d, xp, xp_d, ss, ss_d):

    if x[0]==x[1]:         # outside entire interval 
        xx_d = x_d[-1]

    else:              # interval (s[i-1], s[i])
        ds = s[1]-s[0]
        t  = (ss-s[0]) / ds
        t_d  =  ss_d / ds;
        xx_d = (1   - 3*t**2 + 2*t**3) * x_d[0]     \
             + (      3*t**2 - 2*t**3) * x_d[1]       \
             + (  t - 2*t**2 +   t**3) * ds*xp_d[0] \
             + (    -   t**2 +   t**3) * ds*xp_d[1]   \
             + ( (  - 6*t + 6*t**2) * x[0]          \
               + (    6*t - 6*t**2) * x[1]            \
               + (1 - 4*t + 3*t**2) * ds*xp[0]      \
               + (  - 2*t + 3*t**2) * ds*xp[1] ) * t_d

    return xx_d

def evaluation_xy(x,y,u,output_xy,a,b):
    i = jnp.sum(jnp.where(x<a,1,0))
    j = jnp.sum(jnp.where(y<b,1,0))
    u_x = output_xy[0]
    u_y = output_xy[1]
    u_xy = output_xy[2]
    #splining by fixing y
    if j != 0 and j != x.size:
        u_a2 = spline.eval(x,u[j,:],u_x[j,:],a)
        u_y_2 = spline.eval(x,u_y[j,:],u_xy[j,:],a)
        u_a1 = spline.eval(x,u[j-1,:],u_x[j-1,:],a)
        u_y_1 = spline.eval(x,u_y[j-1,:],u_xy[j-1,:],a)
        y_interval = y[j-1:j+1]
    elif j == 0:
        u_a1 = spline.eval(x,u[j,:],u_x[j,:],a)
        u_y_1 = spline.eval(x,u_y[j,:],u_xy[j,:],a)
        u_a2 = u_a1
        u_y_2 = u_y_1
        y_interval = jnp.array([y[j]]*2)
    else:  # j==x.size
        u_a1 = spline.eval(x,u[j-1,:],u_x[j-1,:],a)
        u_y_1 = spline.eval(x,u_y[j-1,:],u_xy[j-1,:],a)
        u_a2 = u_a1
        u_y_2 = u_y_1
        y_interval = jnp.array([y[j-1]]*2)
    #fixing x
    u_ab = eval_2p(y_interval,jnp.array([u_a1,u_a2]),jnp.array([u_y_1,u_y_2]),b)
    return u_ab

def evaluation_xy_reverse(x,y,u,output_xy,a,b, u_a_bar, u_y__bar, a_bar, b_bar, output_xy_bar, u_bar, u_ab_bar):
    i = jnp.sum(jnp.where(x<a,1,0))
    j = jnp.sum(jnp.where(y<b,1,0))
        
    u_x = output_xy[0]
    u_y = output_xy[1]
    u_xy = output_xy[2]
    
    u_x_bar = output_xy_bar[0]
    u_y_bar = output_xy_bar[1]
    u_xy_bar = output_xy_bar[2]
    
    #splining by fixing y
    if j != 0 and j != x.size:
        u_a2 = spline.eval(x,u[j,:],u_x[j,:],a)
        u_y_2 = spline.eval(x,u_y[j,:],u_xy[j,:],a)
        u_a1 = spline.eval(x,u[j-1,:],u_x[j-1,:],a)
        u_y_1 = spline.eval(x,u_y[j-1,:],u_xy[j-1,:],a)
        y_interval = y[j-1:j+1]
    elif j == 0:
        u_a1 = spline.eval(x,u[j,:],u_x[j,:],a)
        u_y_1 = spline.eval(x,u_y[j,:],u_xy[j,:],a)
        u_a2 = u_a1
        u_y_2 = u_y_1
        y_interval = jnp.array([y[j]]*2)
    else:  # j==x.size
        u_a1 = spline.eval(x,u[j-1,:],u_x[j-1,:],a)
        u_y_1 = spline.eval(x,u_y[j-1,:],u_xy[j-1,:],a)
        u_a2 = u_a1
        u_y_2 = u_y_1
        y_interval = jnp.array([y[j-1]]*2)
    #fixing x
    u_a_bar, u_y__bar, b_bar = \
    eval_2p_reverse(y_interval,jnp.array([u_a1,u_a2]),jnp.array([u_y_1,u_y_2]),b,u_ab_bar, u_a_bar, u_y__bar,b_bar)
    if j != 0 and j != x.size:
        x_b, xp_b, ss_b = eval_reverse(x,u_y[j-1,:], u_y_bar[j-1,:], u_xy[j-1,:], u_xy_bar[j-1,:], a, a_bar, u_y__bar[0])
        u_y_bar = u_y_bar.at[j-1,:].set(x_b)
        u_xy_bar = u_xy_bar.at[j-1,:].set(xp_b)
        a_bar = ss_b
        x_b, xp_b, ss_b = eval_reverse(x,u[j-1,:], u_bar[j-1,:], u_x[j-1,:], u_x_bar[j-1,:], a, a_bar, u_a_bar[0])
        u_bar = u_bar.at[j-1,:].set(x_b)
        u_x_bar = u_x_bar.at[j-1,:].set(xp_b)
        a_bar = ss_b
        x_b, xp_b, ss_b = eval_reverse(x,u_y[j,:], u_y_bar[j,:], u_xy[j,:], u_xy_bar[j,:], a, a_bar, u_y__bar[1])
        u_y_bar = u_y_bar.at[j,:].set(x_b)
        u_xy_bar = u_xy_bar.at[j,:].set(xp_b)
        a_bar = ss_b
        x_b, xp_b, ss_b = eval_reverse(x,u[j,:], u_bar[j,:], u_x[j,:], u_x_bar[j,:], a, a_bar, u_a_bar[1])
        u_bar = u_bar.at[j,:].set(x_b)
        u_x_bar = u_x_bar.at[j,:].set(xp_b)
        a_bar = ss_b
    elif j == 0:
        u_y__bar = u_y__bar.at[0].set(u_y__bar[1])
        u_a_bar = u_a_bar.at[0].set(u_a_bar[1])
        x_b, xp_b, ss_b = eval_reverse(x,u_y[j,:], u_y_bar[j,:], u_xy[j,:], u_xy_bar[j,:], a, a_bar, u_y__bar[1])
        u_y_bar = u_y_bar.at[j,:].set(x_b)
        u_xy_bar = u_xy_bar.at[j,:].set(xp_b)
        a_bar = ss_b
        x_b, xp_b, ss_b = eval_reverse(x,u[j,:], u_bar[j,:], u_x[j,:], u_x_bar[j,:], a, a_bar, u_a_bar[1])
        u_bar = u_bar.at[j,:].set(x_b)
        u_x_bar = u_x_bar.at[j,:].set(xp_b)
        a_bar = ss_b
    else:
        u_y__bar = u_y__bar.at[1].set(u_y__bar[0])
        u_a_bar = u_a_bar.at[1].set(u_a_bar[0])
        x_b, xp_b, ss_b = eval_reverse(x,u_y[j-1,:], u_y_bar[j-1,:], u_xy[j-1,:], u_xy_bar[j-1,:], a, a_bar, u_y__bar[0])
        u_y_bar = u_y_bar.at[j-1,:].set(x_b)
        u_xy_bar = u_xy_bar.at[j-1,:].set(xp_b)
        a_bar = ss_b
        x_b, xp_b, ss_b = eval_reverse(x,u[j-1,:], u_bar[j-1,:], u_x[j-1,:], u_x_bar[j-1,:], a, a_bar, u_a_bar[0])
        u_bar = u_bar.at[j-1,:].set(x_b)
        u_x_bar = u_x_bar.at[j-1,:].set(xp_b)
        a_bar = ss_b
    
    output_xy_bar_new = (u_x_bar, u_y_bar, u_xy_bar)
    
    return u_bar, output_xy_bar_new, a_bar, b_bar

def evaluation_xy_forward(x,y,u,output_xy,a,b,u_d,output_xy_d,a_d,b_d):
    i = jnp.sum(jnp.where(x<a,1,0))
    j = jnp.sum(jnp.where(y<b,1,0))
    
    u_x = output_xy[0]
    u_y = output_xy[1]
    u_xy = output_xy[2]
    
    u_x_d = output_xy_d[0]
    u_y_d = output_xy_d[1]
    u_xy_d = output_xy_d[2]
    
    #splining by fixing y
    if j != 0 and j != x.size:
        u_a2 = spline.eval(x,u[j,:],u_x[j,:],a)
        u_y_2 = spline.eval(x,u_y[j,:],u_xy[j,:],a)
        u_a1 = spline.eval(x,u[j-1,:],u_x[j-1,:],a)
        u_y_1 = spline.eval(x,u_y[j-1,:],u_xy[j-1,:],a)
        #(s, x, x_d, xp, xp_d, ss, ss_d)
        u_a2_d = eval_forward(x,u[j,:],u_d[j,:],u_x[j,:],u_x_d[j,:],a,a_d)
        u_y_2_d = eval_forward(x,u_y[j,:],u_y_d[j,:],u_xy[j,:],u_xy_d[j,:],a,a_d)
        u_a1_d = eval_forward(x,u[j-1,:],u_d[j-1,:],u_x[j-1,:],u_x_d[j-1,:],a,a_d)
        u_y_1_d = eval_forward(x,u_y[j-1,:],u_y_d[j-1,:],u_xy[j-1,:],u_xy_d[j-1,:],a,a_d)
        y_interval = y[j-1:j+1]
    elif j == 0:
        u_a1 = spline.eval(x,u[j,:],u_x[j,:],a)
        u_y_1 = spline.eval(x,u_y[j,:],u_xy[j,:],a)
        u_a2 = u_a1
        u_y_2 = u_y_1
        u_a1_d = eval_forward(x,u[j,:],u_d[j,:],u_x[j,:],u_x_d[j,:],a,a_d)
        u_y_1_d = eval_forward(x,u_y[j,:],u_y_d[j,:],u_xy[j,:],u_xy_d[j,:],a,a_d)
        u_a2_d = u_a1_d
        u_y_2_d = u_y_1_d
        y_interval = jnp.array([y[j]]*2)
    else:  # j==x.size
        u_a1 = spline.eval(x,u[j-1,:],u_x[j-1,:],a)
        u_y_1 = spline.eval(x,u_y[j-1,:],u_xy[j-1,:],a)
        u_a2 = u_a1
        u_y_2 = u_y_1
        u_a1_d = eval_forward(x,u[j-1,:],u_d[j-1,:],u_x[j-1,:],u_x_d[j-1,:],a,a_d)
        u_y_1_d = eval_forward(x,u_y[j-1,:],u_y_d[j-1,:],u_xy[j-1,:],u_xy_d[j-1,:],a,a_d)
        u_a2_d = u_a1_d
        u_y_2_d = u_y_1_d
        y_interval = jnp.array([y[j-1]]*2)
    #fixing x
    u_ab_d = eval_2p_forward(y_interval,jnp.array([u_a1,u_a2]),jnp.array([u_a1_d,u_a2_d]),\
                             jnp.array([u_y_1,u_y_2]),jnp.array([u_y_1_d,u_y_2_d]),b,b_d)
    return u_ab_d


def evaluation_yx(x,y,u,output_yx,a,b):
    i = jnp.sum(jnp.where(x<a,1,0))
    j = jnp.sum(jnp.where(y<b,1,0))
    u_x = output_yx[0]
    u_y = output_yx[1]
    u_xy = output_yx[2]
    #splining by fixing x
    if i != 0 and i != y.size:
        u_a2 = spline.eval(y,u[:,i],u_y[:,i],b)
        u_x_2 = spline.eval(y,u_x[:,i],u_xy[:,i],b)
        u_a1 = spline.eval(y,u[:,i-1],u_y[:,i-1],b)
        u_x_1 = spline.eval(y,u_x[:,i-1],u_xy[:,i-1],b)
        x_interval = x[i-1:i+1]
    elif i == 0:
        u_a1 = spline.eval(y,u[:,i],u_y[:,i],b)
        u_x_1 = spline.eval(y,u_x[:,i],u_xy[:,i],b)
        u_a2 = u_a1
        u_x_2 = u_x_1
        x_interval = jnp.array([x[i]]*2)
    else:  # i==y.size
        u_a1 = spline.eval(y,u[:,i-1],u_y[:,i-1],b)
        u_x_1 = spline.eval(y,u_x[:,i-1],u_xy[:,i-1],b)
        u_a2 = u_a1
        u_x_2 = u_x_1
        x_interval = jnp.array([x[i-1]]*2)
    #fixing y
    u_ab = eval_2p(x_interval,jnp.array([u_a1,u_a2]),jnp.array([u_x_1,u_x_2]),a)
    return u_ab

In [148]:
N = 6
s    = jnp.linspace(0.0,1.0,N)  # spline points
x    = random.normal(sub,(N,))
x_d  = random.normal(sub,(N,))
x_b  = random.normal(sub,(N,))
xp_b = random.normal(sub,(N,))

xp   = spline.derivs(s, x)
xp_d = spline.derivs(s, x_d)    

ss   = np.random.rand()  # uniform on (0,1)
ss_d = np.random.rand()
xx_b = np.random.rand()

x_b  = jnp.zeros_like(x)
xp_b = jnp.zeros_like(x)
ss_b = 0.

xx_d = eval_2p_forward(s, x, x_d, xp, xp_d, ss, ss_d)
x_b, xp_b, ss_b = eval_2p_reverse(s, x, xp, ss, xx_b, x_b, xp_b, ss_b)
print('adj error: %g \n' % (xx_b*xx_d-ss_b*ss_d-dp(x_b,x_d)-dp(xp_b,xp_d)))

adj error: -2.25514e-17 



In [503]:
output_xy = creation_xy(s1, s2, f)
evaluation_xy(s1,s2,f,output_xy,ss_1,ss_2)

Array(1.02552223, dtype=float64)

In [512]:
# Forward mode 
u_d = jnp.ones_like(f)  
output_xy_d = (u_d, u_d, u_d)
forward_primal, forward_d = jvp(evaluation_xy, (s1, s2, f, output_xy, ss_1, ss_2), \
                                      (jnp.zeros_like(x), jnp.zeros_like(y), u_d, output_xy_d, 0.0, 0.0))

# Reverse mode (VJP)
_, vjp_fun = vjp(evaluation_xy, s1, s2, f, output_xy, ss_1, ss_2)
u_b = jnp.ones_like(forward_primal)
reverse_b = vjp_fun(u_b)

print('spline evaluation, forward/reverse sensitivity check')
print('forward sensitivity: %f'%(forward_d))
print('reverse sensitivity: %f'% (sum([dp(jnp.ones_like(f),reverse_b[2])]+[dp(jnp.ones_like(f),i) for i in reverse_b[3]])))
print('adj error: %g \n' % (sum([dp(jnp.ones_like(f),reverse_b[2])]+[dp(jnp.ones_like(f),i) for i in reverse_b[3]]) \
                            - dp(u_b,forward_d)))

spline evaluation, forward/reverse sensitivity check
forward sensitivity: 1.031015
reverse sensitivity: 1.031015
adj error: 0 



In [469]:
forward_d

Array(1., dtype=float64)

In [470]:
reverse_b[3]

(Array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float64),
 Array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float64),
 Array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float64))

In [471]:
reverse_b[4], reverse_b[5]

(Array(0., dtype=float64), Array(0., dtype=float64))

In [514]:
# Forward mode 
u_d = jnp.ones_like(f)  
output_xy_d = (u_d, u_d, u_d)
a_d,b_d = 0.,0. 
forward_d = evaluation_xy_forward(s1,s2,f,output_xy,ss_1,ss_2,u_d,output_xy_d,a_d,b_d)

# Reverse mode (VJP)
u_a_bar = jnp.array([0.,0.])
u_y__bar = jnp.array([0.,0.])
a_bar, b_bar, u_ab_bar = 0., 0., 1.
output_xy_bar = (jnp.zeros_like(f), jnp.zeros_like(f), jnp.zeros_like(f))
u_bar = jnp.zeros_like(f)
reverse_b = evaluation_xy_reverse(s1,s2,f,output_xy,ss_1,ss_2, \
                                  u_a_bar, u_y__bar, a_bar, b_bar, output_xy_bar, u_bar, u_ab_bar)

print('spline evaluation, forward/reverse sensitivity check')
print('forward sensitivity: %f'%(forward_d))
print('reverse sensitivity: %f'% (sum([dp(jnp.ones_like(f),reverse_b[0])]+[dp(jnp.ones_like(f),i) for i in reverse_b[1]])))
print('adj error: %g \n' % (sum([dp(jnp.ones_like(f),reverse_b[0])]+[dp(jnp.ones_like(f),i) for i in reverse_b[1]]) \
                            - dp(u_ab_bar,forward_d)))

spline evaluation, forward/reverse sensitivity check
forward sensitivity: 1.031015
reverse sensitivity: 1.031015
adj error: 0 



In [484]:
forward_d

Array(1., dtype=float64)

In [485]:
reverse_b[1]

(Array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float64),
 Array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float64),
 Array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float64))

In [486]:
reverse_b[2], reverse_b[3]

(Array(0., dtype=float64), Array(0., dtype=float64))

In [477]:
output_yx = creation_yx(s1, s2, f)
evaluation_yx(s1,s2,f,output_yx,ss_1,ss_2)

Array(0.35150377, dtype=float64)

In [464]:
# Forward mode 
u_d = jnp.ones_like(f)  
output_yx_d = (u_d, u_d, u_d)
forward_primal, forward_d = jvp(evaluation_yx, (s1, s2, f, output_yx, ss_1, ss_2), \
                                      (jnp.zeros_like(x), jnp.zeros_like(y), u_d, output_yx_d, 0.0, 0.0))

# Reverse mode (VJP)
_, vjp_fun = vjp(evaluation_yx, s1, s2, f, output_yx, ss_1, ss_2)
u_b = jnp.ones_like(forward_primal)
reverse_b = vjp_fun(u_b)

print('spline evaluation, forward/reverse sensitivity check')
print('adj error: %g \n' % (sum([dp(jnp.ones_like(f),reverse_b[2])]+[dp(jnp.ones_like(f),i) for i in reverse_b[3]]) \
                            - dp(u_b,forward_d)))

spline evaluation, forward/reverse sensitivity check
adj error: 0 



# local vol PDE

In [205]:
import numpy as np
import matplotlib.pyplot as plt
# import call_options as calls
import jax_trid as trid
import spline

In [289]:
# eval from x to y
# dot product function for vectors and arrays

def dp(A_b,A_d):
    return np.sum(np.multiply(A_b,A_d))

#
# define BS discretisation with local volatility
#


def BS_discretisation(J,n,dt,dS, r, s,t,sig,sigp):
    J = int(J)
    a = jnp.zeros(J+1);  b = jnp.zeros(J+1);  c = jnp.zeros(J+1)
    
    for j in range(J):
        sigma = evaluation_xy(s,t,sig, sigp, j*dS, n*dt)
#         print(sigma)
        a = a.at[j].set(dt*( 0.5*sigma**2*j*j - 0.5*r*j     ))
        b = b.at[j].set(dt*(   - sigma**2*j*j           - r ))
        c = c.at[j].set(dt*( 0.5*sigma**2*j*j + 0.5*r*j     ))

    a = a.at[J].set(dt*( -r*J     ))   # enforce right-hand b.c.
    b = b.at[J].set(dt*(  r*J - r ))

    return trid.assemble(a,b,c)

In [497]:
def BS_discretisation_forward(J,n,dt,dS, r, s,t,sig,sigp,r_d, sig_d, sigp_d, ss_d, tt_d):
    a_d = jnp.zeros(J+1);  b_d = jnp.zeros(J+1);  c_d = jnp.zeros(J+1)
    
    for j in range(J):
        sigma = evaluation_xy(s,t,sig, sigp, j*dS, n*dt)
        #(x,y,u,output_xy,a,b,u_d,output_xy_d,a_d,b_d)
        sigma_d = evaluation_xy_forward(s,t,sig, sigp, j*dS, n*dt, sig_d, sigp_d, ss_d, tt_d)
#         print(sigma_d)
        a_d = a_d.at[j].set(dt*(     sigma*sigma_d*j*j - 0.5*r_d*j     ))
        b_d = b_d.at[j].set(dt*(-2.0*sigma*sigma_d*j*j           - r_d ))
        c_d = c_d.at[j].set(dt*(     sigma*sigma_d*j*j + 0.5*r_d*j     ))

    a_d = a_d.at[J].set(dt*( -r_d*J       ))   # enforce right-hand b.c.
    b_d = b_d.at[J].set(dt*(  r_d*J - r_d ))

    return trid.assemble(a_d,b_d,c_d)

def BS_discretisation_reverse(J,n,dt,dS, r, s,t,sig,sigp,r_b, ss_b, tt_b, sig_b, sigp_b, D_b):
    a_b, b_b, c_b = trid.disassemble(D_b)
    
    for j in range(J-1,-1,-1):
        sigma = evaluation_xy(s,t,sig, sigp, j*dS, n*dt)
        sigma_b = dt*sigma*j*j*( a_b[j] - 2.0*b_b[j] + c_b[j] )
        #(x,y,u,output_xy,a,b, u_a_bar, u_y__bar, a_bar, b_bar, output_xy_bar, u_bar, u_ab_bar)
        sig_b, sigp_b, ss_b, tt_b = \
            evaluation_xy_reverse(s,t,sig,sigp, j*dS, n*dt,\
                                jnp.array([0.,0.]),jnp.array([0.,0.]),ss_b,tt_b, sigp_b, sig_b, sigma_b)
        r_b = r_b + dt*( - 0.5*j*a_b[j] - b_b[j] + 0.5*j*c_b[j] )

    r_b = r_b + dt*( - J*a_b[J] + (J - 1.0)*b_b[J] )

    return r_b, sig_b, sigp_b, ss_b, tt_b

In [498]:
r  = 0.05
T  = 1.   # maturity
S0 = 100. # S_0
K  = 100. # strike

M = 4
s = jnp.linspace(0,2*S0,M,dtype=jnp.float64)
t = jnp.linspace(0,T,M,dtype=jnp.float64)
sig  = random.normal(sub,(M,M),dtype=jnp.float64)
sigp = creation_xy(s,t,sig)

J  = 64
N  = 5
dS = 2*S0/J
dt = T/N
n = 1

ss_d = 0.
tt_d = 0.
sig_d = jnp.ones_like(sig)
sigp_d = (sig_d,sig_d,sig_d)
r_d = 1.
D_d = BS_discretisation_forward(J,n,dt,dS, r, s,t,sig,sigp,r_d, sig_d, sigp_d, ss_d, tt_d)

r_b = 0.
ss_b = 0.
tt_b = 0.
sig_b = jnp.zeros_like(sig)
sigp_b = (sig_b,sig_b,sig_b)
D_b = jnp.ones_like(D_d)
BS_discretisation_reverse(J,n,dt,dS, r, s,t,sig,sigp,r_b, ss_b, tt_b, sig_b, sigp_b, D_b)

(Array(-13., dtype=float64),
 Array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float64),
 (Array([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]], dtype=float64),
  Array([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]], dtype=float64),
  Array([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]], dtype=float64)),
 Array(0., dtype=float64),
 Array(0., dtype=float64))

In [323]:
#BS_discretisation(J,n,dt,dS, r, s,t,sig,sigp)
# Forward mode 
sig_d = jnp.ones_like(sig)
sigp_d = (sig_d,sig_d,sig_d)
D_primal, D_d = jvp(BS_discretisation, (float(J),float(n),dt,dS, r, s,t,sig,sigp), \
                        (0.,0.,0.,0.,1.,jnp.zeros_like(s), jnp.zeros_like(t), sig_d,sigp_d))

# Reverse mode (VJP)
_, vjp_fun = vjp(BS_discretisation, J,n,dt,dS, r, s,t,sig,sigp)
D_b = jnp.ones_like(D_primal)
reverse_bbbb = vjp_fun(D_b)
print('RHS:%f'%(dp(D_d,D_b)))
print('LHS:%f'%(r_d*reverse_bbbb[4]+sum([dp(jnp.ones_like(f),reverse_bbbb[7])]+[dp(jnp.ones_like(f),i) for i in reverse_bbbb[8]])))

RHS:-13.000000
LHS:-13.000000


In [516]:
r  = 0.05
T  = 1   # maturity
S0 = 100 # S_0
K  = 100 # strike

M = 10
s = jnp.linspace(0,2*S0,M)
t = jnp.linspace(0,T,M)
sig  = random.normal(sub,(M,M))
sigp = creation_xy(s,t,sig)

J  = 64
N  = 16
dS = 2*S0/J
dt = T/N


def local_vol_PDE(r, s, t, sig, sigp):
    S = jnp.linspace(0,2*S0,J+1) #  np.arange(J+1)*dS
    u = jnp.maximum(S-K,0)

    u_copy = jnp.zeros((J+1,N+1))  # save for reverse mode
    u_copy = u_copy.at[:,N].set(u)
    I   = trid.identity(J+1)

    for n in range(N-1,-1,-1):
        D   = BS_discretisation(J,n,dt,dS, r, s,t,sig,sigp)

        A   = I - 0.5*D
        B   = I + 0.5*D
        v   = trid.multiply(B,u)
        u   = trid.solve(A, v)
        u_copy = u_copy.at[:,n].set(u)

    P   = u[J//2]
    return P

# "forward" mode  (actually backward in time)
r_d    = np.random.randn()
sig_d  = random.normal(sub,(M,M))
sigp_d = (random.normal(sub,(M,M)),random.normal(sub,(M,M)),random.normal(sub,(M,M)))

P_primal, P_d = jvp(local_vol_PDE, (r, s, t, sig, sigp), \
                                      (r_d, jnp.zeros_like(s), jnp.zeros_like(t), sig_d, sigp_d))

print('\nforward sensitivity %f ' % P_d)

# "reverse" mode  (actually forward in time)
_, vjp_fun = vjp(local_vol_PDE, r, s, t, sig, sigp)
P_b = jnp.ones_like(P_primal)
reverse_b = vjp_fun(P_b)

print('reverse sensitivity %f ' % \
      (r_d*reverse_b[0] + sum([dp(sig_d,reverse_b[3])]+\
                                 [dp(i,j) for i,j in zip(sigp_d,reverse_b[4])])))


forward sensitivity 50.894858 
reverse sensitivity 50.894858 


In [517]:
reverse_b[0] #r_b

Array(42.67614718, dtype=float64, weak_type=True)

In [518]:
reverse_b[3] #sig_b

Array([[-8.94085026e-14, -1.92354167e-09,  1.07587742e-02,
         3.54304501e-01,  1.02712992e+00,  2.06939062e+00,
         6.97979096e-01,  1.34008060e-01, -1.91632756e-03,
        -1.13855211e-03],
       [ 4.60910127e-11, -5.37718482e-05, -3.10530860e-02,
        -5.59662073e-01, -2.51223166e-01, -2.24215146e-01,
        -2.06536853e-01, -2.56968161e-01, -7.63395370e-02,
        -3.88447101e-02],
       [-2.09652986e-09, -5.50147911e-03, -2.33648746e-01,
        -6.33648299e-01, -3.90589738e-01,  6.20679361e-02,
        -1.22472878e-01,  2.66052213e-02, -2.55618064e-02,
        -1.68144300e-03],
       [-4.08636575e-08, -3.87422540e-03, -3.53498729e-02,
         1.46958653e-01,  3.91772540e-01,  6.29959465e-01,
         5.07566774e-02,  5.86204095e-01, -7.16396019e-03,
        -4.61476969e-05],
       [ 3.51428292e-08,  1.07258753e-03,  7.83552348e-02,
         2.64525413e-01,  2.00657929e-01,  2.18310393e-01,
         2.35031141e-01,  3.42048550e-02,  1.30574326e-03,
        -5.

In [519]:
def local_vol_PDE_forward(r, s, t, sig, sigp, u_d, r_d, sig_d, sigp_d, s_d, t_d):
    S = jnp.linspace(0,2*S0,J+1) #  np.arange(J+1)*dS
    u = jnp.maximum(S-K,0)

    u_copy = jnp.zeros((J+1,N+1))  # save for reverse mode
    u_copy = u_copy.at[:,N].set(u)
    u_d_copy = u_d.copy()
    I   = trid.identity(J+1)

    for n in range(N-1,-1,-1):
        D   = BS_discretisation(J,n,dt,dS, r, s,t,sig,sigp)
        D_d = BS_discretisation_forward(J,n,dt,dS, r, s,t,sig,sigp,r_d, sig_d, sigp_d, s_d, t_d)


        A   = I - 0.5*D
        B   = I + 0.5*D
        A_d =   - 0.5*D_d
        B_d =   + 0.5*D_d
        v   = trid.multiply(B,u)
        v_d = trid.multiply(B,u_d) + trid.multiply(B_d,u)
        u   = trid.solve(A, v)
        u_d = trid.solve(A, v_d - trid.multiply(A_d,u))
        u_copy = u_copy.at[:,n].set(u)

    P_d = u_d[J//2]
    return P_d

def local_vol_PDE_reverse(r, s, t, sig, sigp,u_b, r_b, ss_b, tt_b, sig_b, sigp_b, P_b):
    S = jnp.linspace(0,2*S0,J+1) #  np.arange(J+1)*dS
    u = jnp.maximum(S-K,0)

    u_copy = jnp.zeros((J+1,N+1))  # save for reverse mode
    u_copy = u_copy.at[:,N].set(u)
    I   = trid.identity(J+1)
    A_list = []
    B_list = []
    

    for n in range(N-1,-1,-1):
        D   = BS_discretisation(J,n,dt,dS, r, s,t,sig,sigp)
        A   = I - 0.5*D
        B   = I + 0.5*D
        A_list.append(A)
        B_list.append(B)
        v   = trid.multiply(B,u)
        u   = trid.solve(A, v)
        u_copy = u_copy.at[:,n].set(u)
    
    u_b = jnp.zeros_like(S)
    u_b = u_b.at[J//2].set(P_b)

    u = u_copy[:,0]
    for n in range(0,N):
        A = A_list[N-n-1]
        B = B_list[N-n-1]
        A_b = np.zeros_like(A)
        B_b = np.zeros_like(B)
        D_b = jnp.zeros_like(D_d)
        v_b = trid.solve(trid.transpose(A), u_b)
        A_b = A_b - trid.outer(v_b,u)
        u = u_copy[:,n+1]
        B_b = B_b + trid.outer(v_b,u)
        u_b = trid.multiply(trid.transpose(B), v_b)
        D_b = - 0.5*A_b + 0.5*B_b
        tt_b = 0.
        r_b, sig_b, sigp_b, ss_b, tt_b = \
        BS_discretisation_reverse(J,n,dt,dS, r, s,t,sig,sigp,r_b, ss_b, tt_b, sig_b, sigp_b, D_b)
    
    return r_b, sig_b, sigp_b

In [520]:
u_d = jnp.zeros_like(S)
ss_d = 0.
tt_d = 0.
print('forward sensitivity: ')
local_vol_PDE_forward(r, s, t, sig, sigp, u_d, r_d, sig_d, sigp_d, ss_d, tt_d)

forward sensitivity: 


Array(50.8948583, dtype=float64)

In [521]:
r_b = 0.
ss_b = 0.
tt_b = 0.
sig_b = jnp.zeros_like(sig)
sigp_b = (sig_b,sig_b,sig_b)
P_b = 1.
reverse_b = local_vol_PDE_reverse(r, s, t, sig, sigp,u_b, r_b, ss_b, tt_b, sig_b, sigp_b, P_b)
print('reverse sensitivity %f ' % \
      (r_d*reverse_b[0] + sum([dp(sig_d,reverse_b[1])]+\
                                 [dp(i,j) for i,j in zip(sigp_d,reverse_b[2])])))

reverse sensitivity 50.894858 


In [522]:
reverse_b[0]#r_b

Array(42.67614718, dtype=float64)

In [523]:
reverse_b[1]#sig_b

Array([[-8.94085026e-14, -1.92354167e-09,  1.07587742e-02,
         3.54304501e-01,  1.02712992e+00,  2.06939062e+00,
         6.97979096e-01,  1.34008060e-01, -1.91632756e-03,
        -1.13855211e-03],
       [ 4.60910127e-11, -5.37718482e-05, -3.10530860e-02,
        -5.59662073e-01, -2.51223166e-01, -2.24215146e-01,
        -2.06536853e-01, -2.56968161e-01, -7.63395370e-02,
        -3.88447101e-02],
       [-2.09652986e-09, -5.50147911e-03, -2.33648746e-01,
        -6.33648299e-01, -3.90589738e-01,  6.20679361e-02,
        -1.22472878e-01,  2.66052213e-02, -2.55618064e-02,
        -1.68144300e-03],
       [-4.08636575e-08, -3.87422540e-03, -3.53498729e-02,
         1.46958653e-01,  3.91772540e-01,  6.29959465e-01,
         5.07566774e-02,  5.86204095e-01, -7.16396019e-03,
        -4.61476969e-05],
       [ 3.51428292e-08,  1.07258753e-03,  7.83552348e-02,
         2.64525413e-01,  2.00657929e-01,  2.18310393e-01,
         2.35031141e-01,  3.42048550e-02,  1.30574326e-03,
        -5.

# local vol SDE

In [524]:
s   = jnp.array([ 0. , 1. , 2.  , 3.  ])
t   = jnp.array([ 0. , 0.33, 0.67, 1.])
M = len(s)
sig = random.normal(sub,(M,M))

T  = 1
N  = 10            # number of timesteps
h  = T/N
r  = 0.05
S0 = 1.0
K  = 0.5

sigp   = creation_xy(s, t, sig)
dW = jnp.sqrt(h)*random.normal(sub,(N,))

def local_vol_SDE(S0, h, r, dW, s, t, sig, sigp):

    S = S0

    for n in range(0,N):
        sigma   = evaluation_xy(s,t,sig,sigp, S, n*h)
        S   = S  *( 1. + r*h + sigma*dW[n] )

    P   = jnp.exp(-r*T)*jnp.maximum(0,S-K)
    return P

In [525]:
# "forward" mode  
sig_d  = random.normal(sub,(M,M))
sigp_d = (random.normal(sub,(M,M)),random.normal(sub,(M,M)),random.normal(sub,(M,M)))

P_primal, P_d = jvp(local_vol_SDE, (S0, h, r, dW, s, t, sig, sigp), \
                            (0.,0.,0., jnp.zeros(N),jnp.zeros_like(s), jnp.zeros_like(t), sig_d, sigp_d))

print('\nforward sensitivity %f ' % P_d)

# "reverse" mode  
_, vjp_fun = vjp(local_vol_SDE, S0, h, r, dW, s, t, sig, sigp)
P_b = jnp.ones_like(P_primal)
reverse_b = vjp_fun(P_b)

print('reverse sensitivity %f ' % \
      (sum([dp(sig_d,reverse_b[6])]+[dp(i,j) for i,j in zip(sigp_d,reverse_b[7])])))


forward sensitivity -0.092720 
reverse sensitivity -0.092720 


In [526]:
reverse_b[6] #sig_b

Array([[-2.12418456e-04, -2.67333685e-01, -2.03454826e-06,
         0.00000000e+00],
       [ 5.10282048e-04,  1.58478693e-01,  4.05036881e-03,
         0.00000000e+00],
       [-1.21514782e-02, -1.34063359e-01,  5.06666467e-04,
         0.00000000e+00],
       [-3.89557666e-03, -7.38107212e-02, -1.05293250e-05,
         0.00000000e+00]], dtype=float64)

In [527]:
reverse_b[7] #sigp_b

(Array([[-7.02785332e-05,  3.25034459e-03,  6.71064345e-07,
          0.00000000e+00],
        [ 1.59573756e-04,  1.84750185e-02, -1.31701915e-03,
          0.00000000e+00],
        [-3.81112938e-03,  2.25422230e-02, -1.64786228e-04,
          0.00000000e+00],
        [-1.23293944e-03,  7.64106350e-03,  3.46182923e-06,
          0.00000000e+00]], dtype=float64),
 Array([[-1.60000108e-05, -1.18771489e-02, -2.16580944e-07,
          0.00000000e+00],
        [ 5.91010836e-05,  2.61247538e-02,  2.07227035e-04,
          0.00000000e+00],
        [-2.35248982e-04, -1.00172195e-02, -5.34490646e-05,
          0.00000000e+00],
        [ 3.51454220e-04,  6.59259935e-03,  6.55599478e-07,
          0.00000000e+00]], dtype=float64),
 Array([[-5.29199026e-06,  2.34383249e-04,  7.14358819e-08,
          0.00000000e+00],
        [ 1.91186156e-05,  5.99007109e-04, -6.74119033e-05,
          0.00000000e+00],
        [-7.58066147e-05,  3.66314873e-04,  1.73856724e-05,
          0.00000000e+00],
        [

In [528]:
def local_vol_SDE_forward(S0, h, r, dW, s, t, sig, sigp, sig_d, sigp_d):

    S = S0
    S_d = jnp.zeros_like(S)
    tt_d = 0.

    for n in range(0,N):
        sigma   = evaluation_xy(s,t,sig,sigp, S, n*h)
        sigma_d = evaluation_xy_forward(s,t,sig, sigp, S, n*h, sig_d, sigp_d, S_d, tt_d)
        S_d = S_d*( 1. + r*h + sigma*dW[n] ) + S*sigma_d*dW[n]
        S   = S  *( 1. + r*h + sigma*dW[n] )

    P_d = jnp.exp(-r*T)*jnp.heaviside(S-K,0.5)*S_d
    return P_d

def local_vol_SDE_reverse(S0, h, r, dW, s, t, sig, sigp, sig_b, sigp_b ,P_b):

    S = S0
    S_copy = jnp.zeros(N)

    for n in range(0,N):
        S_copy = S_copy.at[n].set(S)
        sigma   = evaluation_xy(s,t,sig,sigp, S, n*h)
        S   = S  *( 1. + r*h + sigma*dW[n] )

    S_b    = jnp.exp(-r*T)*jnp.heaviside(S-K,0.5)*P_b
    tt_b = 0.
    
    for n in range(N-1,-1,-1):
        S = S_copy[n]
        sigma   = evaluation_xy(s,t,sig,sigp, S, n*h)
        sigma_b = S_b*S*dW[n]
        S_b = S_b*( 1. + r*h + sigma*dW[n] )
        sig_b, sigp_b, S_b, tt_b = \
            evaluation_xy_reverse(s,t,sig,sigp, S, n*h,\
                                jnp.array([0.,0.]),jnp.array([0.,0.]), S_b, tt_b, sigp_b, sig_b, sigma_b)
    
    return sig_b, sigp_b

In [450]:
def local_vol_SDE_reverse(S0, h, r, dW, s, t, sig, sigp, sig_b, sigp_b ,P_b):

    S = S0
    S_copy = jnp.zeros(N)

    for n in range(0,N):
        S_copy = S_copy.at[n].set(S)
        sigma   = evaluation_xy(s,t,sig,sigp, S, n*h)
        S   = S  *( 1. + r*h + sigma*dW[n] )

    S_b    = jnp.exp(-r*T)*jnp.heaviside(S-K,0.5)*P_b
    tt_b = 0.
    
    for n in range(N-1,-1,-1):
        S = S_copy[n]
        sigma   = evaluation_xy(s,t,sig,sigp, S, n*h)
        sigma_b = S_b*S*dW[n]
        S_b = S_b*( 1. + r*h + sigma*dW[n] )
        _, vjp_fun = vjp(evaluation_xy, s,t,sig,sigp, S, n*h)
        reverse_b = vjp_fun(sigma_b)
#         sig_b, sigp_b, S_b, tt_b = \
#             evaluation_xy_reverse(s,t,sig,sigp, S, n*h,\
#                                 jnp.array([0.,0.]),jnp.array([0.,0.]), S_b, tt_b, sigp_b, sig_b, sigma_b)
        sig_b += reverse_b[2]
        sigp_b = (sigp_b[0]+reverse_b[3][0],sigp_b[1]+reverse_b[3][1],sigp_b[2]+reverse_b[3][2])
        S_b += reverse_b[4] 
        tt_b += reverse_b[5]
    
    return sig_b, sigp_b

In [531]:
print('forward sensitivity: ')
local_vol_SDE_forward(S0, h, r, dW, s, t, sig, sigp, sig_d, sigp_d)

forward sensitivity: 


Array(-0.0927205, dtype=float64)

In [532]:
sig_b = jnp.zeros_like(sig)
sigp_b = (sig_b,sig_b,sig_b)
# P_b = 1.
reverse_b = local_vol_SDE_reverse(S0, h, r, dW, s, t, sig, sigp, sig_b, sigp_b ,P_b)
print('reverse sensitivity %f ' % \
      (sum([dp(sig_d,reverse_b[0])]+[dp(i,j) for i,j in zip(sigp_d,reverse_b[1])])))

reverse sensitivity -0.092720 


In [533]:
reverse_b[0] #sig_b

Array([[-2.12418456e-04, -2.67333685e-01, -2.03454826e-06,
         0.00000000e+00],
       [ 5.10282048e-04,  1.58478693e-01,  4.05036881e-03,
         0.00000000e+00],
       [-1.21514782e-02, -1.34063359e-01,  5.06666467e-04,
         0.00000000e+00],
       [-3.89557666e-03, -7.38107212e-02, -1.05293250e-05,
         0.00000000e+00]], dtype=float64)

In [534]:
reverse_b[1] #sigp_b

(Array([[-7.02785332e-05,  3.25034459e-03,  6.71064345e-07,
          0.00000000e+00],
        [ 1.59573756e-04,  1.84750185e-02, -1.31701915e-03,
          0.00000000e+00],
        [-3.81112938e-03,  2.25422230e-02, -1.64786228e-04,
          0.00000000e+00],
        [-1.23293944e-03,  7.64106350e-03,  3.46182923e-06,
          0.00000000e+00]], dtype=float64),
 Array([[-1.60000108e-05, -1.18771489e-02, -2.16580944e-07,
          0.00000000e+00],
        [ 5.91010836e-05,  2.61247538e-02,  2.07227035e-04,
          0.00000000e+00],
        [-2.35248982e-04, -1.00172195e-02, -5.34490646e-05,
          0.00000000e+00],
        [ 3.51454220e-04,  6.59259935e-03,  6.55599478e-07,
          0.00000000e+00]], dtype=float64),
 Array([[-5.29199026e-06,  2.34383249e-04,  7.14358819e-08,
          0.00000000e+00],
        [ 1.91186156e-05,  5.99007109e-04, -6.74119033e-05,
          0.00000000e+00],
        [-7.58066147e-05,  3.66314873e-04,  1.73856724e-05,
          0.00000000e+00],
        [